# TP Recommendation

In [1]:
from movielensutils import load_movielens, objective
import numpy as np
from scipy.sparse import linalg

In [2]:
R, mask = load_movielens("./ml-100k/u.data")
print(R.shape)

FileNotFoundError: [Errno 2] No such file or directory: './ml-100k/u.data'

### Question 1.1
La matrice a la bonne taille. l'option minidata permet de prendre une  portion de la matrice R et mask R est de taille 100x200 si minidata=True.

In [4]:
print(R[0:10, 0:10])
print(mask[0:10,0:10])
# pour savoir combien de films ont été notés, on regarde le nombre de valeur positive dans le mask
print("nombre de films notés %d " % np.sum(mask))
print("nombre de film si tous le monde notait tous les films %d"%(R.shape[0]*R.shape[1]))

[[ 5.  3.  4.  3.  3.  5.  4.  1.  5.  3.]
 [ 4.  0.  0.  0.  0.  0.  0.  0.  0.  2.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 4.  3.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 4.  0.  0.  0.  0.  0.  2.  4.  4.  0.]
 [ 0.  0.  0.  5.  0.  0.  5.  5.  5.  4.]
 [ 0.  0.  0.  0.  0.  0.  3.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  5.  4.  0.  0.  0.]
 [ 4.  0.  0.  4.  0.  0.  4.  0.  4.  0.]]
[[ True  True  True  True  True  True  True  True  True  True]
 [ True False False False False False False False False  True]
 [False False False False False False False False False False]
 [False False False False False False False False False False]
 [ True  True False False False False False False False False]
 [ True False False False False False  True  True  True False]
 [False False False  True False False  True  True  True  True]
 [False False False False False False  True False False False]
 [False False False False False  True  True False False False]
 [

### Question 1.2

Les lignes de la matrice R représente les différent utilisateurs. Les colonnes représentent les différents film. Il y a donc 943 utilisateurs et 1682 films.
On a 100000 films notés sur 1586126.

### Question 1.3
$Q \rightarrow \frac{\rho}{2}||Q - C_{st}||^2$ est une fonction convexe si $\rho>0$
La somme de fonction convexe est convexe
Cependant, $Q,P \rightarrow \frac{1}{2}||1_{K}\circ (R - QP)||_{F}^2$ n'est pas forcément convexe.
En effet, Soit 
$$
h : x,y \rightarrow (xy)^2
$$
en effet, soit $\lambda \in ]0,1[$
$$
h(\lambda(1,0) + (1-\lambda)(0,1))=\lambda^2(1-\lambda)^2 > \lambda h(1,0)+ (1-\lambda)h(0,1)=0
$$
Donc la fonction objective :
$$
Q,P \rightarrow \frac{1}{2}||1_{K}\circ (R - QP)||_{F}^2 + \frac{\rho}{2}||Q||_{F}^2 + \frac{\rho}{2}||P||_{F}^2
$$
n'est pas convexe.

Supposons $Q$ constant.

Soit $$f : P \rightarrow \frac{1}{2}||1_{K}\circ (R - QP)||_{F}^2 + \frac{\rho}{2}||Q||_{F}^2 + \frac{\rho}{2}||P||_{F}^2$$
$1_{K}$ est une application linéaire. Appliquer $1_{K}$ revient à multiplier par une matrice. appelons là $M_K$.
Soit $H \in \mathbf{R}^{|C|\times|I| }$. On a:
$$
f(P+H) =  \frac{1}{2}||M_K (R - QP - QH||_{F}^2 + \frac{\rho}{2}||Q||_{F}^2 + \frac{\rho}{2}||P+H||_{F}^2 \\
$$
or
$$
\frac{1}{2}||M_K (R - QP - QH||_{F}^2 = \frac{1}{2}||M_K (R - QP||^2 + \frac{1}{2}||M_KQH||^2 + <M_K(R-QP),M_KQH> \\
= \frac{1}{2}||M_K (R - QP||^2 + \frac{1}{2}||M_KQH||^2 + <(M_KQ)^T M_k(R-QP),H>
$$
De plus :
$$
\frac{\rho}{2}||P+H||_{F}^2 = \frac{\rho}{2}(||P||^2 + ||Q||^2 2<P,H>)
$$
On en déduit : 
$$
\nabla f = Q^TM_K^TM_K(R-QP)P^T + \rho P
$$
On peut appliquer un calcul très similaire pour Q
Le gradient de la fonction objective a donc cette forme :
$$
(Q^TM_K^TM_K(-R+QP) + \rho P, M_K^TM_K(-R+QP)P^T + \rho Q)
$$
Si on regarde le gradient de la fonction $h$ defini ci-dessus, on remarque que le gradient comporte des termes
quadratique. Le gradient de la fonction objective n'est pas lipchitzien.

## Trouver $P$ quand $Q_0$ fixé

### Question 2.1


$Q_0$ est une matrice positive. Donc $g$ est convexe.
On a :
$$
\nabla g = Q_0^TM_K^TM_K(-R+Q_0P) + \rho P
$$

### Question 2.2

In [24]:
# calcul de la fonction objective et du gradient
u,s,vh = linalg.svds(R)
Q0 = u 
P0 = vh
print(s)

[ 145.87261327  158.21191449  159.15359872  217.84622472  244.83634567
  640.63362257]
